In [2]:
import pandas as pd
import numpy as np
from bq_helper import BigQueryHelper

In [5]:
from blockchain import blockexplorer
from blockchain import util
import time

In [9]:
from google.cloud import bigquery

In [8]:
query = """
SELECT
 count(`hash`)
FROM
  `bigquery-public-data.crypto_bitcoin.blocks` blocks
"""

In [11]:
client = bigquery.Client()
q_job = client.query(query)

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response',))

In [7]:
util.TIMEOUT = 15

In [21]:
# unit in satoshi
address = blockexplorer.get_address('3JfAJhVeDwm8YWPKDNFjvPtyEEwggXwLLB')
address2 = blockexplorer.get_address('3EXxYoB61g13UsH1K678G5zeEE8W3XJ2p3')
address3 = blockexplorer.get_address('1NqdBaU9WCvmjGQ1omPx6NHGJYzQo9udd6')
print(address.total_received)
print(address.total_sent)
print(address.final_balance)
print(address.n_tx)
print(address.transactions[0].hash)

655000000000
654990000000
10000000
43
c7ae6fb55b5919bf8239555a11f0309527bac4f18bdc5b9f98fbcd7bf7e83342


idea: use bfs to expand adr to explore. 

for simplicity, neglect amount of transfer for now

for transacitons going out of Target Address, not so interested right now

problem: how to distinguish incoming or outgoing tx?

timeout: The read operation timed out
possible solution: request API key?

In [18]:
tx = address.transactions[0]
print(tx.inputs[2].address)
print (addr_queue[0])

3JfAJhVeDwm8YWPKDNFjvPtyEEwggXwLLB


In [27]:
address = '3JfAJhVeDwm8YWPKDNFjvPtyEEwggXwLLB' # tyr with 1 with fewer txs
tx_queue = []
addr_queue = []
addr_queue.append(address)
addr_dict = dict() # key:str address, value: number of visits
max_iter = 2
nodes_per_round = 1
for i in range(max_iter):
    print ("iteration :" + str(i))
    if len(addr_queue) == 0:
        print ("max_iter not reached. All possible addresses explored.")
        break;
    for j in range(nodes_per_round):
        nodes_per_round = 0
        address = addr_queue.pop(0)
         # this process is slow
        if address not in addr_dict:
            addr = blockexplorer.get_address(address)
            tx_queue.extend(addr.transactions)
            addr_dict[address] = 1
            print (address)
        else:
            addr_dict[address] += 1
        while len(tx_queue) > 0:
            tx = tx_queue.pop(0)
            isReceiver = False;
            # check if this addr is receiving money
            for out in tx.outputs:
                if addr.address == out.address:
                    isReceiver = True;
                    break;
            if isReceiver:
                for inp in tx.inputs:
                    addr_queue.append(inp.address)
                    nodes_per_round += 1

iteration :0
3JfAJhVeDwm8YWPKDNFjvPtyEEwggXwLLB
iteration :1
3LC3mbk1NowqEFKfyRxh35DAQhvms7pojA
39ACzDZRJBd1STHgvL5AMQvo48iqMoEdSS
3PpXuqu2KG4WZmHyFp2XyU5gmaFDxmxJJ1
3QxkG5HitPoXsjTfogMk4AYqj4TJhJnZGH
3CLon2XMt7yBuVrqtPBw5cW9o93dJeAknM
3E9CYFsCiDtQ4xaxsj1hcZLUh8hwpimioj
3QiLXF8tpa17k9XLZ4rbcoXTKJDoWsNaGG
3MWaY5qReHm3qqnhgYCE437cVD4moWeUys
33SBjyCNDr499kFkg93nxchtRg9ij38H2C
3QEvTTqJ5eCcXYy2p7jJEtqH4xFnyCjXme
37D1iDiquoAW875RXjvPc6HRgsmsLriDbo
3N9R1K7r8veTLK16qZGKpP1CVdyKYBmGsc
3ESJe7jsjhJrHk9dkvvbqYfscRnrZnwSJx
3AzK8em6MeGgZkvDnrzRQS1p1dS4Hkg9KY
3E7DcbGJ3XEzffUCKHQw9HorzLc5NiiANC
3L9M3qzpoQh4jxaj9iNRx3zxgxB23SJFJZ
3NSLeRUGq9pWFuJJjcnvHxRP7SV7g2nNbU
35SUxj432rR5upPTQ3BGUydyrFtktfJhxc
3PvzNLk5aTUcx347gDpG28jrDGjKytqgKg
3M57n3hC8cknGbA1VFmWMaSjjRPRghxrss
335pdkwTWmnZ1ssnQPznw24nNrB3WEghXp
35QLG6yzjbeBv6DKgW2QhMJ5guq76JanxP
3EwkzbWLTch2tP45nAFJsCfyhWF4VcZ1Ba
3CwQJRFzHRzkoMP295PEHt9FKUsN8DHHRt
38GGWEvS8ufUmdQV9EURuShsSbiCgCqsmD
3Na1JeyLibsN2i4NN35MdreKAvDLDUHnDQ
3EBR8JsGLvecQGYbxHf3WkAkeThEQ

In [28]:
print (len(addr_queue))
print (len(tx_queue))
print (len(addr_dict))

1183
0
84


In [29]:
for k,v in addr_dict.items():
    print (k,v)

3JfAJhVeDwm8YWPKDNFjvPtyEEwggXwLLB 1
3LC3mbk1NowqEFKfyRxh35DAQhvms7pojA 1
39ACzDZRJBd1STHgvL5AMQvo48iqMoEdSS 1
3PpXuqu2KG4WZmHyFp2XyU5gmaFDxmxJJ1 1
3QxkG5HitPoXsjTfogMk4AYqj4TJhJnZGH 1
3CLon2XMt7yBuVrqtPBw5cW9o93dJeAknM 1
3E9CYFsCiDtQ4xaxsj1hcZLUh8hwpimioj 1
3QiLXF8tpa17k9XLZ4rbcoXTKJDoWsNaGG 1
3MWaY5qReHm3qqnhgYCE437cVD4moWeUys 1
33SBjyCNDr499kFkg93nxchtRg9ij38H2C 1
3QEvTTqJ5eCcXYy2p7jJEtqH4xFnyCjXme 1
37D1iDiquoAW875RXjvPc6HRgsmsLriDbo 1
3N9R1K7r8veTLK16qZGKpP1CVdyKYBmGsc 1
3ESJe7jsjhJrHk9dkvvbqYfscRnrZnwSJx 1
3AzK8em6MeGgZkvDnrzRQS1p1dS4Hkg9KY 1
3E7DcbGJ3XEzffUCKHQw9HorzLc5NiiANC 1
3L9M3qzpoQh4jxaj9iNRx3zxgxB23SJFJZ 1
3NSLeRUGq9pWFuJJjcnvHxRP7SV7g2nNbU 1
35SUxj432rR5upPTQ3BGUydyrFtktfJhxc 1
3PvzNLk5aTUcx347gDpG28jrDGjKytqgKg 1
3M57n3hC8cknGbA1VFmWMaSjjRPRghxrss 1
335pdkwTWmnZ1ssnQPznw24nNrB3WEghXp 1
35QLG6yzjbeBv6DKgW2QhMJ5guq76JanxP 1
3EwkzbWLTch2tP45nAFJsCfyhWF4VcZ1Ba 1
3CwQJRFzHRzkoMP295PEHt9FKUsN8DHHRt 1
38GGWEvS8ufUmdQV9EURuShsSbiCgCqsmD 1
3Na1JeyLibsN2i4NN35MdreKAvDLDUHnDQ 1
3